In [1]:
import numpy as np
import pandas as pd

In [25]:
"""
Inputs de muestra
"""

fecha_accidente = "2020-03"

sueldos = [
            58000,
            58000,
            68000,
            70000,
            65000,
            65000,
            65000,
            65000,
            65000,
            75000,
            70000,
            70000,           
]
sueldos = list(filter(None, sueldos))
sueldos = list(map(float,sueldos))

edad = 28

incapacidad_actual = 33

tipo_accidente = 'in_itinere'#'in_itinere','en_ocasion'

In [11]:
ripte = pd.read_csv('https://infra.datos.gob.ar/catalog/sspm/dataset/158/distribution/158.1/download/remuneracion-imponible-promedio-trabajadores-estables-ripte-total-pais-pesos-serie-mensual.csv')

In [12]:
"""
Preparación del dataframe general de ripte
"""

ripte['variacion'] = ripte['ripte'].pct_change(periods=1)*100

ripte['indice'] = 100.0

for i in ripte.index:
    if i == 0:
        continue
    ripte.at[i, 'indice'] = ripte.at[i-1,'indice'] * (1+(ripte.at[i,'variacion']/100))

ripte['final'] = 100.0
alto = 100.0
for i in ripte['final'].index:
    if ripte.at[i,'indice'] > alto:
        alto = ripte.at[i,'indice']
    ripte.at[i,'final'] = alto

ripte = ripte.fillna(0)
ripte = ripte.round(2)

ripte['indice_tiempo'] = ripte['indice_tiempo'].astype('datetime64')
ripte = ripte.rename(columns={'indice_tiempo':'fechas'})
ripte['fechas'] = ripte['fechas'].dt.to_period('M')

In [17]:
"""
Creacion y merge del dataframe particular del cliente
"""

cliente = pd.DataFrame(sueldos)
cliente = cliente.rename(columns={0:'sueldos'})

cliente['fechas'] = pd.date_range(end=fecha_accidente,freq='M',periods=len(sueldos))
cliente['fechas'] = cliente['fechas'].dt.to_period('M')

cliente = pd.merge(cliente, ripte, on='fechas', how='left')
cliente = pd.merge(cliente, tasas, on="fechas", how="left")

indice = ripte[ripte['fechas']==fecha_accidente].final
indice = float(indice)

cliente['coeficiente'] = 0.0

cliente = cliente.assign(coeficiente = lambda x: (indice/x['final']))
cliente = cliente.assign(actualizados = lambda x: (x['sueldos']*x['coeficiente']))
cliente = cliente.round(2)

In [18]:
"""
Resultado final
"""

resultado = cliente['actualizados'].mean().round(2)

In [15]:
"""
Cargando las tasas de una pagina random porque al BNA le chupa un huevo
"""

link = 'http://www.abogadosrafaela.com.ar/serviciosaprof/tasas-activas-banco-nacion/'

tasas1 = pd.read_html(link, index_col=0, header=0, decimal=','
    )[0].astype(float)

tasas1 = tasas1.drop(columns='Unnamed: 5')

tasas1['2019']['Enero'] = tasas1['2019']['Enero'] * 10000

tasas1 = tasas1.apply(lambda x: x/10000)

link = 'http://www.abogadosrafaela.com.ar/serviciosaprof/tasas-activas-banco-nacion/'

tasas2 = pd.read_html(link, index_col=0, header=0, decimal=','
    )[1].astype(float)

tasas2['2015']['Noviembre'] = tasas2['2015']['Noviembre'] * 10000
tasas2['2016']['Marzo'] = tasas2['2016']['Marzo'] * 10000
tasas2['2016']['Septiembre'] = tasas2['2016']['Septiembre'] * 10000
tasas2['2018']['Octubre'] = tasas2['2018']['Octubre'] * 10000
tasas2['2018']['Noviembre'] = tasas2['2018']['Noviembre'] * 10000

tasas2 = tasas2.apply(lambda x: x/10000)

tasas = pd.concat([tasas2,tasas1], axis=1)

tasas = tasas.drop(columns=['2014','2015','2016'])

ultimo = int(tasas.columns.max())+1

tasas3 = pd.DataFrame(pd.date_range(start='2017-01',end=str(ultimo),freq='M'))
tasas3[0] = tasas3[0].dt.to_period('M')

tasas3 = tasas3.rename(columns={0:'fechas'})

df = pd.DataFrame()
for i in tasas.columns:
    dfn = tasas[i]
    df = pd.concat([df,dfn])

df = df.reset_index()

tasas = pd.concat([tasas3,df[0]],axis=1)
tasas = tasas.rename(columns={0:'tasas'})

In [16]:
"""
Hardcodeados los ultimos meses
"""
tasas.loc[66,'tasas'] = 4.75   # Este hay que borrarlo eventualmente
tasas.loc[67,'tasas'] = 5.017

In [19]:
"""
Calculo final con intereses
"""

fecha_calculo_final = cliente['fechas'].max()

interes = tasas[tasas['fechas'] > fecha_calculo_final]['tasas'].sum()/100 + 1

resultado = (resultado * interes).round(2)

resultado_ci = 53 * resultado *  (65/edad) * incapacidad_actual/(100)
resultado_ci = round(resultado_ci,2)

if tipo_accidente == 1:
    resultado_ci_final = round(resultado_ci*1.2,2)
else:
    resultado_ci_final = resultado_ci


In [20]:
resultado

139656.41

In [21]:
resultado_ci

1187755.95

In [23]:
cliente[['fechas','sueldos','indice','actualizados']]

,fechas,sueldos,indice,actualizados
0,2019-11,58000.0,5554.15,77294.45
1,2019-12,65000.0,5666.48,84905.91
2,2020-01,60000.0,6066.07,73211.91
3,2020-02,60000.0,6445.13,68906.07
4,2020-03,63000.0,6500.72,71732.67
5,2020-04,63000.0,6510.18,71628.44
6,2020-05,63000.0,6521.87,71500.05
7,2020-06,72000.0,6670.93,79888.46
8,2020-07,65000.0,6908.52,69641.20
9,2020-08,65000.0,6945.86,69266.82
